In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

In [ ]:
# icml_face_data = pd.read_csv('/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/icml_face_data.csv')
train_df = pd.read_csv('/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/train.csv')
test_df = pd.read_csv('/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/test.csv')
# submission_df = pd.read_csv('/kaggle/input/challenges-in-representation-learning-facial-expression-recognition-challenge/example_submission.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

## Class imbalance

In [ ]:
num_classes = train_df['emotion'].nunique()
print('Number of classes ',num_classes)

In [ ]:
train_df['emotion'].value_counts()

In [ ]:
pixels = np.array([ np.array(i.split(), dtype=float) for i in train_df['pixels'].values])
labels = np.array(train_df['emotion'].values)

In [ ]:
print('Shape of pixel array',pixels.shape)
print('Shape of target labels',labels.shape)

In [ ]:
# lets over sample minority class with smote
over = SMOTE()
X, y = over.fit_resample(pixels, labels)

In [ ]:
from collections import Counter
Counter(y)

In [ ]:
print('Shape of pixel array',X.shape)
print('Shape of target labels',y.shape)

## Utils

In [ ]:
emotions = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

def display_image_and_label(img,label):
    plt.imshow(img.reshape(48,48), cmap=plt.cm.gray)
    plt.title(label)   
    plt.axis('off')
    
def visualize(images, labels, subplot_config={'hspace':0.4}, figsize=(12,10)):
    import random
    a = 0
    b = images.shape[0]

    plt.figure(figsize=figsize)
    for i in range(50):
        r = random.randint(a,b)
        plt.subplot(5,10,i+1)
        display_image_and_label(images[r], emotions[labels[r]])
        plt.subplots_adjust(**subplot_config)    
    

In [ ]:
visualize(X,y)

## Modeling

In [ ]:
simple_model = tf.keras.Sequential([

    tf.keras.layers.Dense(512),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Dense(256),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Dense(128),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Dense(64),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Dense(32),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    
    
    tf.keras.layers.Dense(num_classes)
])

simple_model.compile(optimizer='adam',
                     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                     metrics=['accuracy'])



In [ ]:
np.random.seed(42)
np.random.shuffle(X)
np.random.seed(42)
np.random.shuffle(y)

In [ ]:
import math
def lr_schedule(epoch):
    return 0.01*math.pow(0.7,epoch)
    
lr = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
es = tf.keras.callbacks.EarlyStopping(patience=5)
plateau = tf.keras.callbacks.ReduceLROnPlateau(patience=2)

In [ ]:
simple_model_history = simple_model.fit(X[:-505], y[:-505],
                                        epochs=20,
                                        validation_data=(X[-505:],y[-505:]),
                                        batch_size=64, callbacks=[lr ,es])

## Prediction

In [ ]:
test_pixels = np.array([ np.array(i.split(), dtype=float) for i in test_df['pixels'].values])

In [ ]:
test_pixels.shape

In [ ]:
predicted_labels = simple_model.predict(test_pixels, batch_size=64)

In [ ]:
index = np.argmax(predicted_labels, axis=1)

visualize(test_pixels, index)

In [ ]:
Counter([emotions[i] for i in index])


## Transfer learning

In [ ]:
# pretrained model

pretrained_model = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False, weights='imagenet', input_shape=(48,48,3))

In [ ]:
pretrained_model.trainable=False

Note: each Keras Application expects a specific kind of input preprocessing. For MobileNetV2, call tf.keras.applications.mobilenet_v2.preprocess_input on your inputs before passing them to the model. mobilenet_v2.preprocess_input will scale input pixels between -1 and 1.

In [ ]:
X_ =  tf.keras.applications.mobilenet_v2.preprocess_input(X.copy())

Dataset

In [ ]:
def get_dataset(images, labels, batch_size):
    images = images.reshape(-1,48,48,1)
    ds = tf.data.Dataset.from_tensor_slices((images,labels))
    ds = ds.shuffle(images.shape[0])
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
ds = get_dataset(X_, y, batch_size=64)
train_ds = ds.skip(80)
val_ds = ds.take(80)

custom preprocessing layer 

In [ ]:
class ImageTile(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__(trainable=False)
    
    def call(self,inputs):
        return tf.tile(inputs,tf.constant([1,1,1,3]))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(48,48,1)),
    
    ImageTile(),
    
    pretrained_model,
    
    tf.keras.layers.GlobalMaxPool2D(),
    
    tf.keras.layers.Dense(124),
    
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Activation('relu'),
    
    tf.keras.layers.Dense(num_classes)
    
])


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
import math
def lr_schedule(epoch):
    return 0.01*math.pow(0.7,epoch)
    
lr = tf.keras.callbacks.LearningRateScheduler(lr_schedule)
es = tf.keras.callbacks.EarlyStopping(patience=5)
plateau = tf.keras.callbacks.ReduceLROnPlateau(patience=2)

In [ ]:
history = model.fit(train_ds, epochs=25, validation_data=val_ds, callbacks=[plateau, es])

## prediction

In [ ]:
test_images_ = tf.keras.applications.mobilenet_v2.preprocess_input(test_pixels.copy())
test_ds = tf.data.Dataset.from_tensor_slices(test_images_.reshape(-1,48,48,1)).batch(64)
test_prediction = model.predict(test_ds)

In [ ]:
index = np.argmax(test_prediction,axis=1)

visualize(test_images_,index)

In [ ]:
Counter([emotions[i] for i in index])

## Fine tuning

In [ ]:
print('Total number of layers in our pertrained model :',len(model.layers[1].layers))

In [ ]:
# Trainable variables in pretrained model
model.layers[1].trainable_variables

In [ ]:
# lets make top layers of the pertrained model trainable and retrain the model 

model.layers[1].trainable = True

for i in range(100):
    model.layers[1].layers[i].trainable=False

print('Number of trainable layers in pretrained model',  len(model.layers[1].trainable_variables))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


history = model.fit(train_ds, epochs=25, validation_data=val_ds, callbacks=[plateau, es])

## prediction

In [ ]:
test_prediction = model.predict(test_ds)
index = np.argmax(test_prediction,axis=1)

visualize(test_images_,index)

In [ ]:
Counter([emotions[i] for i in index])

In [ ]:
visualize(test_images_,index)

In [ ]:
visualize(test_images_,index)